# 1. threading 모듈로 쓰레드 구현하기

In [1]:
from threading import Thread
import time

### 1) Thread 구현

In [2]:

def work(id, start, end, result):
    total = 0
    for i in range(start, end):
        total+=i
    result.append(total)
    
if __name__=='__main__':
    START, END = 0, 100000000
    result = list()

    start_time = time.time()

    th1 = Thread(target=work, args=(1,START,END, result))

    th1.start() # 쓰레드 시작
    th1.join() # 쓰레드가 끝날 때까지 기다림

    end_time = time.time()


print(f"Rsult: {sum(result)}")
print("소요 시간: ", end_time - start_time)

Rsult: 4999999950000000
소요 시간:  4.93511700630188


### 2) Multi Thread 구현

In [7]:
def work(id, start, end, result):
    total = 0
    for i in range(start, end):
        total+=i
    result.append(total)
    
if __name__=='__main__':
    START, END = 0, 100000000
    result = list()

    start_time = time.time()

    th1 = Thread(target=work, args=(1,START,END//2, result))
    th2 = Thread(target=work, args=(2,END//2,END, result))

    th1.start() # 쓰레드 시작
    th2.start()
    th1.join() # 쓰레드가 끝날 때까지 기다림
    th2.join()

    end_time = time.time()


print(f"Rsult: {sum(result)}")
print("소요 시간: ", end_time - start_time)

Rsult: 4999999950000000
소요 시간:  5.2463059425354


## 1,2 번 비교

- 2번이 멀티쓰레드를 사용함에도 불구하고 시간이 더 걸림
    - 이유: 파이썬의 GIL 정책 때문임.

## GIL(Global Interpreter Lock)

언어에서 자원을 보호하기 위해 락(lock) 정책을 사용한다.

파이썬에서는 하나의 프로세스 안에 모든 자원의 락을 글로벌하게 관리함으로써 하나의 쓰레드만 자원을 컨트롤하여 동작하도록 한다.

위 코드에서 result라는 자원을 공유하는 두 개의 쓰레드를 동시에 실행시키지만, 결국 GIL 때문에 한 번에 하나의 쓰레드만 계산을 실행하여 실행시간이 비슷한 것이다.

GIL 덕분에 자원관리(ex.가비지 컬렉팅)를 더 쉽게 구현할 수 있었지만, 지금처럼 멀티 코어가 당연한 시대에서는 조금 아쉬운 것이 사실이다.

그렇다고 파이썬 쓰레드가 쓸모 없는 것은 아니다.
GIL이 적용되는 것은 cpu 동작에서이고 쓰레드가 cpu 동작을 마치고 I/O 작업을 실행하는 동안에는 다른 쓰레드가 cpu 동작을 동시에 실행할 수 있다. 따라서 cpu 동작이 많지 않고 I/O동작이 더 많은 프로그램에서는 멀티 쓰레드만으로 성능적으로 큰 효과를 얻을 수 있다.

### 이러한 상황에서 계산을 병렬처리하는데 도움을 주는 것이 바로 multiprocessing 모듈이다.

# 2. mutiprocessing 모듈로 멀티 프로세스 구현하기

In [5]:
from multiprocessing import Process, Queue

In [10]:
def work(id, start, end, result):
    total = 0
    for i in range(start, end):
        total+=i
    result.put(total)
    
if __name__ == "__main__":
    START, END = 0, 10000000
    result = Queue()
    
    start_time = time.time()
    
    th1 = Process(target=work, args=(1, START, END//2, result))
    th2 = Process(target=work, args=(2, END//2, END, result))
    
    th1.start()
    th2.start()
    th1.join()
    th2.join()
    
    end_time = time.time()
    
    result.put('STOP')
    total = 0
    while True:
        tmp = result.get()
        if tmp == 'STOP':
            break
        else:
            total += tmp
    print(f"Result: {total}")

print("소요시간: ", end_time-start_time)

Result: 0
소요시간:  0.1139984130859375


# Thread vs Process

결론적으로 파이썬에서 병렬처리를 구현하는 방식은 두 가지로 멀티 쓰레드와 멀티 프로레스가 있다.

### Thread
쓰레드는 가볍지만 GIL로 인해 계산 처리하는 작업은 한 번에 하나의 쓰레드에서만 작동하여 cpu 작업이 적고 I/O 작업이 많은 병쳘처리 프로그램에서 효과를 볼 수 있다.

### Process
프로세스는 각자 고유한 메모리 영역을 가지기 때문에 더 많은 메모리를 필요로 하디만, 각각 프로세스에서 병렬로 cpu 작업을 할 수 있고 이를 이용해 여러 머신에서 동작하는 분산 처리 프로그래밍도 구현할 수 있다